In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 

data = pd.read_csv('./datasets/ds.csv')
city = gpd.read_file('./datasets/省市县-湖泊/渭河流域地级市region.shp')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59949 entries, 0 to 59948
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Aspect   59949 non-null  float64
 1   Elev     59949 non-null  int64  
 2   Slope    59949 non-null  float64
 3   PreMaxC  59949 non-null  float64
 4   PreMinC  59949 non-null  float64
 5   PreSumC  59949 non-null  float64
 6   TmpMaxC  59949 non-null  float64
 7   TmpMinC  59949 non-null  float64
 8   TmpAvgC  59949 non-null  float64
 9   NtlC     59949 non-null  float64
 10  PdC      59949 non-null  float64
 11  LuC      59949 non-null  float64
 12  FVCC     59949 non-null  float64
 13  Long     59949 non-null  float64
 14  Lat      59949 non-null  float64
dtypes: float64(14), int64(1)
memory usage: 6.9 MB


In [2]:
# from sklearn.preprocessing import MinMaxScaler
# min_max_scaler = MinMaxScaler()


indepent_var = data.columns.to_list()[:12]
repose_var = 'FVCC' 

X = data[indepent_var]
y = data[repose_var]
#X.iloc[:,1:] = min_max_scaler.fit_transform(data[indepent_var[1:]])
#y = min_max_scaler.fit_transform(np.array(data[repose_var]).reshape(-1, 1))
X.head(6)

,Aspect,Elev,Slope,PreMaxC,PreMinC,PreSumC,TmpMaxC,TmpMinC,TmpAvgC,NtlC,PdC,LuC
0,0.000000,2397,22.944578,-0.402857,-1.570677,-7.001504,-0.002331,-0.019850,-0.010138,0.0,-0.205263,1.0
1,0.000000,2466,28.421749,-0.468872,-1.566917,-7.987970,-0.002331,-0.016917,-0.010501,0.0,-0.201504,1.0
2,0.000000,2740,24.026747,-0.337068,-1.363910,-0.645865,-0.029774,-0.047970,-0.043152,0.0,-0.630075,1.0
3,0.000000,2388,23.283827,-0.401654,-1.560902,-2.743609,-0.021504,-0.036090,-0.031310,0.0,-0.129323,1.0
4,0.025615,2221,23.921211,-0.469549,-1.554135,-7.576692,-0.002331,-0.016917,-0.010457,0.0,-0.607519,1.0
5,0.399800,2172,18.808655,-0.500451,-1.554887,-7.614286,-0.002331,-0.015865,-0.010376,0.0,-0.178947,1.0


In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59949 entries, 0 to 59948
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Aspect   59949 non-null  float64
 1   Elev     59949 non-null  int64  
 2   Slope    59949 non-null  float64
 3   PreMaxC  59949 non-null  float64
 4   PreMinC  59949 non-null  float64
 5   PreSumC  59949 non-null  float64
 6   TmpMaxC  59949 non-null  float64
 7   TmpMinC  59949 non-null  float64
 8   TmpAvgC  59949 non-null  float64
 9   NtlC     59949 non-null  float64
 10  PdC      59949 non-null  float64
 11  LuC      59949 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 5.5 MB


In [ ]:
data = gpd.GeoDataFrame(data=data.iloc[:,:12],
                        geometry=gpd.points_from_xy(data.lon,data.lat),
                        crs=4326)
data.head(6)

In [ ]:
import matplotlib.colors as colors

chars=[]
for i in range(97,123):
    chars.append(chr(i))

cdict = ['#A9F090','#40B73F','#63B7FF','#0000FE','#FF00FC','#850042']
pre_cmap = colors.ListedColormap(cdict) 
colormap = ['terrain','gist_earth','terrain',pre_cmap,pre_cmap,pre_cmap,
            'jet','jet','jet','magma_r','magma_r','RdYlGn']

plt.figure(figsize=(16,12))
for i,col in enumerate(indepent_var+['FVCC']):

    ax = plt.subplot(3,4,i+1)
    data.plot(ax=ax,column=col,cmap=colormap[i],markersize = 0.001) 
    city.plot(ax=ax, ec = 'k', fc='none', linewidth = 0.1)    
    im = plt.gca().get_children()[0]
    bar = plt.colorbar(im,ax=ax,location='bottom',pad=0.05,
                       aspect=30,extend='both',shrink=0.8)
    bar.ax.tick_params(axis='x', which='major', direction='in')
    ax.set_title(f'({chars[i]}){col}',fontsize=12,loc='left')
    ax.set_xticks([])
    ax.set_yticks([])
plt.subplots_adjust(top=0.98, bottom=0, right=0.98, left=0.02,wspace=0.01, 
                    hspace=0.03)
plt.margins(0.1, 0.1)
plt.savefig('./figure/variable.png',dpi=600)

### XGBoost建模

In [4]:
import xgboost
import shap
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2,
                                                    random_state = 42)

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

space = {
    'max_depth':hp.choice('max_depth', np.arange(1, 45, 1, dtype=int)),
    'colsample_bytree':hp.quniform('colsample_bytree', 0.1, 1.0, 0.05),
    'min_child_weight':hp.choice('min_child_weight', np.arange(1, 45, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.1, 1.0, 0.05),
    'learning_rate':hp.choice('learning_rate', np.arange(0.05, 1.0, 0.05)),
    'gamma': hp.quniform('gamma', 0, 5, 0.05),
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}

def score(params, n_folds=5):
    
    #Cross-validation
    d_train = xgboost.DMatrix(X,y)
    
    cv_results = xgboost.cv(params, d_train, nfold=n_folds, num_boost_round=1000,
            early_stopping_rounds=10, metrics=['rmse'], seed=42)

    
    loss = min(cv_results['test-rmse-mean'])
    
    return loss


def optimize(trials, space):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=750,
                rstate = np.random.default_rng(42))
    return best

In [5]:
trials = Trials()
best_params = optimize(trials, space)

# 运行最好的参数
best_params = space_eval(space, best_params)
best_params

100%|██████████| 750/750 [2:38:48<00:00, 12.70s/trial, best loss: 0.001979822693588394]   


{'colsample_bytree': 0.8,
 'eval_metric': 'rmse',
 'gamma': 0.0,
 'learning_rate': 0.1,
 'max_depth': 39,
 'min_child_weight': 42,
 'objective': 'reg:squarederror',
 'subsample': 0.9}

In [6]:
# 训练模型
data_test = xgboost.DMatrix(X_test,y_test)
data_train = xgboost.DMatrix(X_train,y_train)
model = xgboost.train(best_params, data_train, 
                      num_boost_round=500, verbose_eval=False,
                      evals=[(data_test, "Test")],
                      early_stopping_rounds=10)

In [7]:
# 保存模型
import pickle
pickle.dump(model, open("./result/xgboost_shap/model.second.dat", "wb"))

In [8]:
# 加载已经训练好的模型
import pickle
model = pickle.load(open("./result/xgboost_shap/model.second.dat", "rb"))

In [9]:
import xgboost
import shap
from sklearn.metrics import mean_squared_error, r2_score

data_all = xgboost.DMatrix(X)
y_pred = model.predict(data_all)
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)
print(f'XGBoost模型RMSE {rmse}')
print(f'XGBoost模型R方 {r2}')

XGBoost模型RMSE 0.0012543172523548306
XGBoost模型R方 0.8691209591971689


In [10]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression().fit(X,y)

print(f'LM模型RMSE {np.sqrt(mean_squared_error(y, lm.predict(X)))}')
print(f'LM模型R方 {r2_score(y, lm.predict(X))}')

LM模型RMSE 0.003009031090462741
LM模型R方 0.2468028603916752


### SHAP解释

In [11]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap_interaction_values = explainer.shap_interaction_values(X)

np.save('./result/xgboost_shap/shap_values',shap_values)
np.save('./result/xgboost_shap/shap_interaction_values',shap_interaction_values)

In [12]:
shap_values = np.load('./result/xgboost_shap/shap_values.npy',allow_pickle=True)
shap_interaction_values = np.load('./result/xgboost_shap/shap_interaction_values.npy',allow_pickle=True)
pd.DataFrame(data=shap_interaction_values[:,np.arange(12), np.arange(12)],
             columns=X.columns.to_list()).to_csv('./result/xgboost_shap/shap_independent_values.csv',
            index=False)
pd.DataFrame(data=shap_values,
             columns=X.columns.to_list()).to_csv('./result/xgboost_shap/shap_interaction_values.csv',
             index=False)

In [ ]:
# Independent feature SHAP value order
shap.summary_plot(shap_interaction_values[:,np.arange(15), np.arange(15)], 
                  X,plot_type= "bar",show=False)
plt.savefig('./figure/xgboost_shap/Independent-feature-SHAP-value-order.png',dpi=600)

In [ ]:
# Interaction feature SHAP value order
shap.summary_plot(shap_values, X ,plot_type= "bar",show = False)
plt.savefig('./figure/xgboost_shap/Interaction-feature-SHAP-value-order.png',dpi=600)

In [ ]:
# Independent feature SHAP value order
shap.summary_plot(shap_interaction_values[:,[i for i in range(11)],[i for i in range(11)]], 
                  X,plot_type= "violin",show=False)
plt.savefig('./figure/xgboost_shap/Independent-Feature-order.png',dpi=600)

In [ ]:
# Interaction feature SHAP value order
shap.summary_plot(shap_values, X ,plot_type= "violin",show = False)
plt.savefig('./figure/xgboost_shap/Interaction-Feature-order.png',dpi=600)

In [ ]:
# 按SHAP值绝对值等于0 小于等于0.00001 小于等于0.0001查找SHAP值正负号改变时样本值
import pandas as pd 
threshold_range = pd.DataFrame()

a = 0
for i,v in enumerate(indepent_var):
    index = np.abs(shap_interaction_values[:,i,i])==0
    if np.sum(index) == 0.0:
        print(f'-------------- {v} 不存在等于 0 的 SHAP 值--------------') 
        index = np.abs(shap_interaction_values[:,i,i])<=0.00001
        if np.sum(index) == 0.0:
            index = np.abs(shap_interaction_values[:,i,i])<=0.0001
    threshold_value = data[v][index]
    min_t_value = np.min(threshold_value)
    max_t_value = np.max(threshold_value)
    threshold_range[v] = [min_t_value,max_t_value]
    print(f"{v}'s threshold_value {min_t_value:.4f} to {max_t_value:.4f}")
    
threshold_range.to_excel('./result/xgboost_shap/Independent_threshold_range.xlsx')
threshold_range

In [ ]:
# Feature Independent SHAP value
plt.figure(figsize=(25,15))
for i,col in enumerate(indepent_var):
    ax = plt.subplot(3,5,i+1)
    ax.scatter(data[col], shap_interaction_values[:, i, i], color='grey',s=0.3)
    ax.axhline(0,color='r', linestyle='--',)
    ax.axvline(threshold_range[col][0],color='black', linestyle='--',)
    ax.axvline(threshold_range[col][1],color='black', linestyle='--',)
    ax.set_xlabel(col,fontsize=13)
    if (i == 0) or (i ==5) or (i==10):
        ax.set_ylabel("Feature Independent SHAP value",fontsize=15)
plt.savefig('./figure/xgboost_shap/Feature-Independent-SHAP-Value.png',dpi=600)

In [ ]:
# 按SHAP值绝对值等于0 小于等于0.00001 小于等于0.0001查找SHAP值正负号改变时样本值
import pandas as pd 
threshold_range = pd.DataFrame()

a = 0
for i,v in enumerate(indepent_var):
    index = np.abs(shap_values[:, i])==0
    if np.sum(index) == 0.0:
        print(f'-------------- {v} 不存在等于 0 的 SHAP 值--------------') 
        index = np.abs(shap_values[:, i])<=0.00001
        if np.sum(index) == 0.0:
            index = np.abs(shap_values[:, i])<=0.0001
    threshold_value = data[v][index]
    min_t_value = np.min(threshold_value)
    max_t_value = np.max(threshold_value)
    threshold_range[v] = [min_t_value,max_t_value]
    print(f"{v}'s threshold_value {min_t_value:.4f} to {max_t_value:.4f}")
threshold_range.to_excel('./result/xgboost_shap/Interaction_threshold_range.xlsx')
threshold_range

In [ ]:
# Interaction Feature SHAP Value
plt.figure(figsize=(25,15))
for i,col in enumerate(indepent_var):
    ax = plt.subplot(3,5,i+1)
    ax.scatter(data[col], shap_values[:, i], color='grey',s=0.3)
    ax.axhline(0,color='r', linestyle='--',)
    ax.axvline(threshold_range[col][0],color='black', linestyle='--',)
    ax.axvline(threshold_range[col][1],color='black', linestyle='--',)
    ax.set_xlabel(col,fontsize=13)
    if (i == 0) or (i ==5) or (i==10):
        ax.set_ylabel("Feature Interaction SHAP value",fontsize=15)
plt.savefig('./figure/xgboost_shap/Interaction-Feature-SHAP-Value.png',dpi=600)

In [ ]:
# Indenpendent feature map
chars=[]
for i in range(97,123):
    chars.append(chr(i))
    
plt.figure(figsize=(20,13.5))
for i,col in enumerate(indepent_var):
    ax = plt.subplot(3,4,i+1)
    city.plot(ax=ax, ec = 'k', fc='none', linewidth = 0.5)
    data.assign(cl=shap_interaction_values[:,i,i]) \
        .plot(ax=ax,column='cl',
              cmap=shap.plots.colors.red_white_blue,
              vmin=np.min(shap_interaction_values[:,i,i]), 
              vmax=np.max(shap_interaction_values[:,i,i]),
              markersize = 0.001)
    im = plt.gca().get_children()[0]
    bar = plt.colorbar(im,ax=ax,location='bottom',pad=0.05,
                       aspect=30,extend='both',shrink=0.8)
    bar.ax.tick_params(axis='x', which='major', direction='in')
    ax.set_title(f'({chars[i]}){col}',fontsize=15,loc='left')
    ax.set_xticks([])
    ax.set_yticks([])
plt.subplots_adjust(top=0.98, bottom=0, right=0.98, left=0.02,wspace=0.01, 
                    hspace=0.03)
plt.margins(0.1, 0.1)
plt.savefig('./figure/xgboost_shap/Indenpendent-Feature-Map.png',dpi=600)

In [ ]:
# Interaction feature map
plt.figure(figsize=(20,13.5))
for i,col in enumerate(indepent_var):
    ax = plt.subplot(3,4,i+1)
    city.plot(ax=ax, ec = 'k', fc='none', linewidth = 0.5) 
    data.assign(cl=shap_values[:, i]) \
        .plot(ax=ax,column='cl',
            cmap=shap.plots.colors.red_white_blue,
            markersize = 0.001)
       
    im = plt.gca().get_children()[0]
    bar = plt.colorbar(im,ax=ax,location='bottom',pad=0.05,
                       aspect=30,extend='both',shrink=0.8)
    bar.ax.tick_params(axis='x', which='major', direction='in')
    ax.set_title(f'({chars[i]}){col}',fontsize=15,loc='left')
    ax.set_xticks([])
    ax.set_yticks([])
plt.subplots_adjust(top=0.98, bottom=0, right=0.98, left=0.02,wspace=0.01, 
                    hspace=0.03)
plt.margins(0.1, 0.1)
plt.savefig('./figure/xgboost_shap/Interaction-Feature-Map.png',dpi=600)

In [ ]:
for i in indepent_var:
    for j in indepent_var:
        if i == j :
            continue
        else:
            shap.dependence_plot(i, shap_values, X, interaction_index=j,show=False)
            plt.savefig(f'./figure/xgboost_shap/feature_dependence/{i}-{j}.png',dpi=600)